# Genk network - non-linear model

## Imports

In [1]:
import modesto.main
import modesto.utils as ut
from pkg_resources import resource_filename
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib notebook

## Optimization settings

In [2]:
horizon = 5*3600
time_step = 60

n_neighs = 3

start_time = pd.Timestamp('20140101')
n_steps = int(horizon/time_step)

## Neighborhood data

In [3]:
neighs = ['WaterscheiGarden', 
          'ZwartbergNEast', 
          'ZwartbergNWest', 
          'ZwartbergSouth', 
          'OudWinterslag', 
          'Winterslag',
          'Boxbergheide', 
          'TermienEast', 
          'TermienWest']

In [4]:
all_pipes = ['dist_pipe{}'.format(i) for i in range(14)]

diams = {1: [350, 0, 350],
         2: [400, 250, 350],
         3: [450, 250, 350, 200, 200],
         4: [500, 250, 350, 350, 200, 300],
         5: [500, 250, 350, 350, 400, 300, 200, 200],
         6: [600, 250, 350, 500, 200, 300, 400, 200, 350, 0, 350],
         7: [700, 250, 350, 600, 200, 300, 500, 200, 500, 400, 350],
         8: [700, 250, 350, 600, 200, 300, 500, 200, 500, 400, 350, 200, 200],
         9: [700, 250, 350, 600, 200, 300, 600, 200, 500, 400, 350, 300, 200, 250]}

diameters = diams[n_neighs]

### Network graph

In [ ]:
 g = nx.DiGraph()

g.add_node('Producer', x=5000, y=5000, z=0,
           comps={'plant': 'Plant'})
g.add_node('p1', x=3500, y=6100, z=0,
           comps={})
if n_neighs >= 1:
    g.add_node('WaterscheiGarden', x=3500, y=5100, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 2:
    g.add_node('ZwartbergNEast', x=3300, y=6700, z=0,
               comps={'building': 'SubstationepsNTU',})
                     # 'DHW': 'BuildingFixed'})
if n_neighs >= 3:
    g.add_node('p2', x=1700, y=6300, z=0,
               comps={})
    g.add_node('ZwartbergNWest', x=1500, y=6600, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 4:
    g.add_node('ZwartbergSouth', x=2000, y=6000, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 5:
    g.add_node('p3', x=250, y=5200, z=0,
               comps={})
    g.add_node('OudWinterslag', x=1700, y=4000, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 6:
    g.add_node('p4', x=0, y=2700, z=0,
               comps={})
    g.add_node('Winterslag', x=1000, y=2500, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 7:
    g.add_node('Boxbergheide', x=-1200, y=2100, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 8:
    g.add_node('p5', x=620, y=700, z=0,
               comps={})
    g.add_node('TermienEast', x=800, y=880, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})
if n_neighs >= 9:
    g.add_node('TermienWest', x=0, y=0, z=0,
               comps={'building': 'SubstationepsNTU',})
                      # 'DHW': 'BuildingFixed'})

In [ ]:
pipes = []

if n_neighs >= 1:
    g.add_edge('Producer', 'p1', name='dist_pipe0')
    g.add_edge('p1', 'WaterscheiGarden', name='dist_pipe2')
    pipes.append('dist_pipe0')
    pipes.append('dist_pipe2')
if n_neighs >= 2:
    g.add_edge('p1', 'ZwartbergNEast', name='dist_pipe1')
    pipes.append('dist_pipe1')
if n_neighs >= 3:
    g.add_edge('p1', 'p2', name='dist_pipe3')
    g.add_edge('p2', 'ZwartbergNWest', name='dist_pipe4')
    pipes.append('dist_pipe3')
    pipes.append('dist_pipe4')
if n_neighs >= 4:
    g.add_edge('p2', 'ZwartbergSouth', name='dist_pipe5')
    pipes.append('dist_pipe5')
if n_neighs >= 5:
    g.add_edge('p2', 'p3', name='dist_pipe6')
    g.add_edge('p3', 'OudWinterslag', name='dist_pipe7')
    pipes.append('dist_pipe6')
    pipes.append('dist_pipe7')
if n_neighs >= 6:
    g.add_edge('p3', 'p4', name='dist_pipe8')
    g.add_edge('p4', 'Winterslag', name='dist_pipe10')
    pipes.append('dist_pipe8')
    pipes.append('dist_pipe10')
if n_neighs >= 7:
    g.add_edge('p4', 'Boxbergheide', name='dist_pipe9')
    pipes.append('dist_pipe9')
if n_neighs >= 8:
    g.add_edge('p4', 'p5', name='dist_pipe11')
    g.add_edge('p5', 'TermienEast', name='dist_pipe12')
    pipes.append('dist_pipe11')
    pipes.append('dist_pipe12')
if n_neighs >= 9:
    g.add_edge('p5', 'TermienWest', name='dist_pipe13')
    pipes.append('dist_pipe13')

## The modesto object

In [ ]:
optmodel = modesto.main.Modesto(pipe_model='FiniteVolumePipe', graph=g, temperature_driven=True)
optmodel.opt_settings(allow_flow_reversal=False)

## Load data

In [ ]:
heat_profile = ut.read_time_data(resource_filename(
        'modesto', 'Data/HeatDemand'), name='TEASER_GenkNET_per_neighb.csv')

t_amb = ut.read_time_data(resource_filename('modesto', 'Data/Weather'), name='extT.csv')['Te']
t_g = pd.Series(12 + 273.15, index=t_amb.index)

wd = ut.read_time_data(resource_filename('modesto', 'Data'), name='Weather/weatherData.csv')
QsolN = wd['QsolN']
QsolE = wd['QsolE']
QsolS = wd['QsolS']
QsolW = wd['QsolW']

c_f = ut.read_time_data(path=resource_filename('modesto', 'Data'), name='ElectricityPrices/DAM_electricity_prices-2014_BE.csv')['price_BE']

mults = ut.read_file(resource_filename(
    'modesto', 'Data/HeatDemand'), name='TEASER_number_of_buildings.csv', timestamp=False).loc['Number of buildings']

## Parameters

In [ ]:
general_params = {'Te': t_amb,
                  'Tg': t_g,
                  'Q_sol_E': QsolE,
                  'Q_sol_W': QsolW,
                  'Q_sol_S': QsolS,
                  'Q_sol_N': QsolN,
                  'time_step': time_step,
                  'horizon': horizon,
                  'elec_cost': c_f}

optmodel.change_params(general_params)

In [ ]:
for n in range(n_neighs):
    neigh = neighs[n]
    building_params = {
        'mult': mults[neigh],
        'heat_flow': heat_profile[neigh] / mults[neigh],
        'temperature_radiator_in': 47 + 273.15,
        'temperature_radiator_out': 35 + 273.15,
        'temperature_supply_0': 60 + 273.15,
        'temperature_return_0': 40 + 273.15,
        'lines': ['supply', 'return'],
        'thermal_size_HEx': 15000,
        'exponential_HEx': 0.7,
        'mf_prim_0': 0.2
    }

    optmodel.change_params(building_params, node=neigh, comp='building')

In [ ]:
for i, pipe in enumerate(pipes):
    pipe_params = {'diameter': diameters[all_pipes.index(pipe)],
                   'max_speed': 3,
                   'Courant': 1,
                   'Tg': pd.Series(12+273.15, index=t_amb.index),
                   'Tsup0': 57+273.15,
                   'Tret0': 40+273.15,
                   }

    optmodel.change_params(pipe_params, comp=pipe)

In [ ]:
prod_design = {'efficiency': 1,
               'PEF': 1,
               'CO2': 0.178,  # based on HHV of CH4 (kg/KWh CH4)
               'fuel_cost': pd.Series([0.25] * int(n_steps/2) + [0.5] * (n_steps - int(n_steps/2))),
               # http://ec.europa.eu/eurostat/statistics-explained/index.php/Energy_price_statistics (euro/kWh CH4)
               'Qmax': 1.5e12,
               'ramp_cost': 0,
               'CO2_price': c_f,
               'temperature_max': 90 + 273.15,
               'temperature_min': 57 + 273.15,
               'temperature_supply_0': 65 + 273.15,
               'temperature_return_0': 30 + 273.15,
               'heat_estimate': heat_profile['WaterscheiGarden'] + heat_profile['ZwartbergNEast']}

optmodel.change_params(prod_design, 'Producer', 'plant')

## Compilation and solving

In [ ]:
compile_order = [['Producer', None],
                 ['Producer', 'plant']]

if n_neighs >= 1:
    compile_order.insert(0, [None, 'dist_pipe0'])
    compile_order.insert(0, ['p1', None])
    compile_order.insert(0, [None, 'dist_pipe2'])
if n_neighs >= 2:
    compile_order.insert(0, [None, 'dist_pipe1'])
if n_neighs >= 3:
    compile_order.insert(0, [None, 'dist_pipe3'])
    compile_order.insert(0, ['p2', None])
    compile_order.insert(0, [None, 'dist_pipe4'])
if n_neighs >= 4:
    compile_order.insert(0, [None, 'dist_pipe5'])
if n_neighs >= 5:
    compile_order.insert(0, [None, 'dist_pipe6'])
    compile_order.insert(0, ['p3', None])
    compile_order.insert(0, [None, 'dist_pipe7'])
if n_neighs >= 6:
    compile_order.insert(0, [None, 'dist_pipe8'])
    compile_order.insert(0, ['p4', None])
    compile_order.insert(0, [None, 'dist_pipe10'])
if n_neighs >= 7:
    compile_order.insert(0, [None, 'dist_pipe9'])
if n_neighs >= 8:
    compile_order.insert(0, [None, 'dist_pipe11'])
    compile_order.insert(0, ['p5', None])
    compile_order.insert(0, [None, 'dist_pipe12'])
if n_neighs >= 9:
    compile_order.insert(0, [None, 'dist_pipe13'])
for n in range(n_neighs):
    compile_order.insert(0, [neighs[n], None])
    compile_order.insert(0, [neighs[n], 'building'])

In [ ]:
optmodel.compile(start_time=start_time,
                 compile_order=compile_order)

optmodel.set_objective('cost')

optmodel.solve(tee=True, mipgap=0.2, last_results=False, g_describe=[480])

dist_pipe0 has 11 volumes, one element has a length of 169.10068397943886
dist_pipe2 has 6 volumes, one element has a length of 166.66666666666666
dist_pipe1 has 4 volumes, one element has a length of 158.11388300841898
dist_pipe3 has 11 volumes, one element has a length of 164.64336614795303
dist_pipe4 has 3 volumes, one element has a length of 120.18504251546631


## Collect results

In [ ]:
# Heat flows and mass flows
prod_hf = optmodel.get_result('heat_flow', node='Producer', comp='plant')
prod_mf = optmodel.get_result('mass_flow', node='Producer', comp='plant')
neigh_hf = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])
neigh_mf = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])

for n in range(n_neighs):
    neigh = neighs[n]
    mult = mults[neigh]
    neigh_hf[neigh] = (optmodel.get_result('heat_flow', node=neigh, comp='building')*mult)
    neigh_mf[neigh] = (optmodel.get_result('mf_prim', node=neigh, comp='building')*mult)

In [ ]:
# Temperatures
prod_T_sup = optmodel.get_result('Tsup', node='Producer', comp='plant') - 273.15
prod_T_ret = optmodel.get_result('Tret', node='Producer', comp='plant') - 273.15
neigh_T_sup = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])
neigh_T_ret = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])
slack = pd.DataFrame(columns=[neighs[i] for i in range(n_neighs)])

for n in range(n_neighs):
    neigh = neighs[n]
    mult = mults[neigh]
    neigh_T_sup[neigh] = (optmodel.get_result('Tpsup', node=neigh, comp='building') - 273.15)
    neigh_T_ret[neigh] = (optmodel.get_result('Tpret', node=neigh, comp='building') - 273.15)
    # slack[neigh] = optmodel.get_result('hf_slack', node=neigh, comp='building')

In [ ]:
prod_e = sum(prod_hf)
neigh_e = neigh_hf.sum(axis=0)

# Efficiency
print('\nNetwork')
print('Efficiency', (sum(neigh_e)) / (prod_e + 0.00001) * 100, '%')

In [ ]:
fig= plt.figure()
ax = fig.add_subplot(111)
ax.plot(prod_hf, label='Producer')
ax.plot(neigh_hf.sum(axis=1), label='Users and storage')
ax.axhline(y=0, linewidth=2, color='k', linestyle='--')
ax.set_title('Heat flows [W]')
ax.legend()
fig.tight_layout()

In [ ]:
fig1, axarr = plt.subplots(1, 1)
axarr.plot(prod_mf, label='Producer')
for neigh in [neighs[i] for i in range(n_neighs)]:
    axarr.plot(neigh_mf[neigh], label=neigh)
axarr.plot(neigh_mf.sum(axis=1), label='all users')
axarr.set_title('Mass flows network')
axarr.legend()
fig1.tight_layout()

In [ ]:
fig2, axarr = plt.subplots(1, 1)
axarr.plot(prod_T_sup, label='Producer Supply', color='r')
axarr.plot(prod_T_ret, label='Producer Return', linestyle='--', color='r')
for neigh in [neighs[i] for i in range(n_neighs)]:
    axarr.plot(neigh_T_sup[neigh], label='{} Supply'.format(neigh))
    axarr.plot(neigh_T_ret[neigh], label='{} Return'.format(neigh), linestyle='--')
axarr.legend()
axarr.set_title('Network temperatures')
fig2.tight_layout()